# 第17章 新しいデータの合成 ― 敵対的生成ネットワーク

## 17.1 敵対的生成ネットワーク

## 17.2 敵対的生成ネットワークを一から実装する

### 17.2.2 生成器ネットワークと識別器ネットワークを実装する

生成器ネットワーク用と識別器ネットワーク用に2つのヘルパー関数を定義する。

In [ ]:
import tensorflow_datasets as tfds
import numpy as np

## 生成器ネットワークの関数を定義
def make_generator_network(num_hidden_layers=1, num_hidden_units=100,
                          num_output_units=784):
    model = tf.keras.Sequential()
    for i in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(units=num_hidden_units, use_bias=False))
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Dense(units=num_output_units, activation="tanh"))
        return model

## 識別器ネットワークの関数を定義
def make_disctiminator_network(num_hidden_layers=1, num_hidden_units=100,
                              num_output_units=1):
    model = tf.keras.Sequential()
    for i in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(units=num_hidden_units))
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Dropout(rate=0.5))

    model.add(tf.keras.layers.Dense(units=num_outputs, activation=None)))
    return model

次に、モデルを訓練するための設定を指定する。<br>
MNISTデータセットの画像のサイズは28x28である。<br>
さらに、入力ベクトル$z$のサイズを20に指定し、モデルの重みを初期化するために一様分布の乱数を使う。

In [ ]:
image_size = (28, 28)
z_size = 20
mode_z = "uniform"
gen_hidden_layers = 1
gen_hidden_size = 100
disc_hidden_layers = 1
disc_hidden_size = 100
tf.random.set_seed(1)
gen_model = make_generator_network(num_hidden_layers=gen_hidden_layers,
                                  num_hidden_units=gen_hidden_size,
                                  num_output_units=np.prod(image_size))
gen_model.build(input_shape=(None, z_size))
gen_model.summary()

disc_model = make_disctiminator_network(num_hidden_layers=disc_hidden_layers,
                                        num_hidden_units=disc_hidden_size)
disc.model.build(input_shape=(None, np.prod(image_size)))
disc.model.summary()

In [ ]:
disc_model = make_discriminator_network(num_hidden_layers=disc_hidden_layers,
                                       num_hidden_units=disc_hidden_size)
disc_model.build(input_shape=(None, np.prod(image_size)))
disc_model.summary()

### 17.2.3 訓練データセットを定義する

In [ ]:
mnist_bldr = tfds.builder("mnist")
mnist_bldr.download_and_prepare()
mnist = mnist_bldr.as_dataset(shuffle_files=False)

def preprocess(ex, mode="uniform"):
    image = ex["image"]
    image = tf.image.convert_image_dtypr(image, tf.float32)
    image = tf.reshape(image, [-1])
    image = image*2 -1.0
    if mode == "uniform":
        input_z = tf.random.uniform(shape=(z_size,), minval=-1.0, maxval=1.0)
    elif mode == "normal":
        input_z = tf.random.normal(shape=(z_size,))
    else:
        return input_z, image

mnist_trainset = mnist["train"]
mnist_trainset = mnist_trainset.map(preprocess)

In [ ]:
mnist_trainset = mnist_trainset.batch(32, drop_remainder=True)
input_z, input_real = next(iter(mnist_trainset)))
print("input-z -- shape: ", input_z.shape)
print("input-real -- shape: ", input_real.shape)

In [ ]:
g_output = gen_model(input_z)
print("Output of G -- shape:", g_output.shape)

In [ ]:
d_logits_real = disc_model(input_real)
d_logits_fake = disc_model(g_model)
print("Disc. (real) -- shape:", d_logits_real.shape)
print("Disc. (fake) -- shape:", d_logits_fake.shape)

2つのロジット`d_logits_fake`と`d_logits_real`は、このモデルを訓練するための損失関数の計算に使われる

### 17.2.4 GANモデルを訓練する

次に、損失関数として`BinaryCrossentropy`クラスのインスタンスを作成し、前項で処理したバッチに関連する生成器ネットワークと識別器ネットワークの損失項を計算する

In [ ]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
## 生成器ネットワークの損失関数
g_labels_real = tf.ones_like(d_logits_fake)
g_loss = loss_fn(y_true=g_labels_real, y_pred=d_logits_fake)
print("Generator Loss: {:.4f}".format(g_loss))

## 識別器ネットワークの損失関数
d_labels_real = tf.ones_like(d_logits_real)
d_labels_fake = tf.zeros_like(d_logits_fake)
d_loss_real = loss_fn(y_true=d_labels_real, y_pred=d_logits_real)
d_loss_fake = loss_fn(y_true=d_labels_fake, y_pred=d_logits_fake)
print("Discriminator Losses: Real {:.4f} Fake {:.4f}".format(d_loss_real.numpy(), d_loss_fake.numpy()))

`tf.GradientTape`を使ってモデルの重みについて損失項の勾配を計算し、2つの独立したAdamオプティマイザを使って生成器ネットワークと識別器ネットワークを交互に訓練する。

In [ ]:
import time

num_epochs = 100
batch_size = 64
image_size = (28, 28)
z_size = 20
mode_z = "uniform"
gen_hidden_layers = 1
gen_hidden_size = 100
disc_hidden_layers = 1
disc_hidden_size = 100

tf.random.set_seed(1)
np.random.seed(1)

if mode_z == "uniform":
    fixed_z = tf.random.uniform(shape=(batch_size, z_size), minval=-1, maxval=1)
elif mode_z == "normal":
    fixed_z = tf.random.uniform(shape=(batch_size, z_size))

def create_samples(g_model, input_z):
    g_output = g_model(input_z, training=False)
    images = tf.reshape(g_output, (batch_size, *image_size))
    return (images+1)/2.0

## データセットの準備
mnist_trainset = mnist["train"]
mnist_trainset = mnist_trainset.map(lambda ex: preprocess(ex, mode=mode_z))
mnist_trainset = mnist_trainset.shuffle(10000)
mnist_trainset = mnist_trainset.batch(batch_size, drop_remainder=True)

## モデルの準備
with tf.device(device_name):
    gen_model = make_generator_network(num_hidden_layers=gen_hidden_layers,
                                       num_hidden_units=gen_hidden_size,
                                       num_output_units=np.prod(image_size))
    gen_model.build(input_shape=(None, z_size))
    disc_model = make_disctiminator_network(num_hidden_layers=disc_hidden_layers,
                                            num_hidden_units=disc_hidden_size)
    disc_model.build(input_shape=(None, np.prod(image_size)))